In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

2024-01-15 19:28:00.916076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.__version__)
# 20231225 應該是當初安裝時程式有異，所以keras沒抓到tensorflow預設的版本
# 20240104 tf2.10 version can run
print(keras.__version__)

2.12.0
2.12.0


In [3]:
# Listing 6.1 Word-level one-hot encoding (toy example)
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
token_index = {}

for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
            
max_length = 10
results = np.zeros(shape=(len(samples), max_length, max(token_index.values()) + 1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.


In [4]:
# Listing 6.2 Character-level one-hot encoding (toy example)
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable
token_index = dict(zip(range(1, len(characters) + 1), characters))
max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.keys()) + 1))

for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = token_index.get(character)
        results[i, j, index] = 1.
        

In [5]:
# Listing 6.3 Using Keras for word-level one-hot encoding
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))


Found 9 unique tokens.


In [6]:
# Listing 6.4 Word-level one-hot encoding with hashing trick (toy example)
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
dimensionality = 1000
max_length = 10
results = np.zeros((len(samples), max_length, dimensionality))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.
        

In [16]:
# Listing 6.5 Instantiating an Embedding layer
# ref: https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64)
print(type(embedding_layer))

<class 'keras.layers.core.embedding.Embedding'>


In [11]:
# Listing 6.6 Loading the IMDB data for use with an Embedding layer
from keras.datasets import imdb
# 20240115 新版已將該函數移至它處
#from keras import preprocessing
from keras.utils import pad_sequences

max_features = 10000
maxlen = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

#x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
#x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [12]:
print(x_train)

[[  65   16   38 ...   19  178   32]
 [  23    4 1690 ...   16  145   95]
 [1352   13  191 ...    7  129  113]
 ...
 [  11 1818 7561 ...    4 3586    2]
 [  92  401  728 ...   12    9   23]
 [ 764   40    4 ...  204  131    9]]


In [13]:
# Listing 6.7 Using an Embedding layer and classifier on the IMDB data
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 8)             80000     
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


2024-01-15 21:50:34.587281: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6779 - acc: 0.5964 - val_loss: 0.6429 - val_acc: 0.6778
Epoch 2/10
625/625 [==============================] - 2s 2ms/step - loss: 0.5654 - acc: 0.7423 - val_loss: 0.5432 - val_acc: 0.7212
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4740 - acc: 0.7804 - val_loss: 0.5095 - val_acc: 0.7408
Epoch 4/10
625/625 [==============================] - 2s 2ms/step - loss: 0.4273 - acc: 0.8065 - val_loss: 0.5024 - val_acc: 0.7474
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3964 - acc: 0.8220 - val_loss: 0.5004 - val_acc: 0.7516
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3712 - acc: 0.8368 - val_loss: 0.5035 - val_acc: 0.7528
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3495 - acc: 0.8492 - val_loss: 0.5094 - val_acc: 0.7516
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0.

In [17]:
# Listing 6.8 Processing the labels of the raw IMDB data
import os

# Linux path
imdb_dir = '/home/earvin/workspaces/datasets/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                 labels.append(1)

In [19]:
print(texts)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# Listing 6.9 Tokenizing the text of the raw IMDB data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]